## Setup

In [1]:
from dask_yarn import YarnCluster
from dask.distributed import Client

/home/hadoop/miniconda/lib/python3.8/site-packages/dask_yarn/core.py:16: FutureWarning: format_bytes is deprecated and will be removed in a future release. Please use dask.utils.format_bytes instead.
  from distributed.utils import (
/home/hadoop/miniconda/lib/python3.8/site-packages/dask_yarn/core.py:16: FutureWarning: parse_timedelta is deprecated and will be removed in a future release. Please use dask.utils.parse_timedelta instead.
  from distributed.utils import (


In [2]:
import pandas as pd
import numpy as np
import time

In [3]:
# Create a cluster where each worker has 4 vCPU core and 8 GiB of memory:
cluster = YarnCluster(environment="/home/hadoop/environment.tar.gz",
                      worker_vcores = 4,
                      worker_memory = "8GiB"
                      )

# Scale cluster out to 2 such workers:
cluster.scale(2)

# Connect to the cluster (before proceeding, you should wait for workers to be registered by the dask scheduler, as below):
client = Client(cluster)

distributed.scheduler - INFO - Clear task state
distributed.scheduler - INFO -   Scheduler at:   tcp://172.31.18.95:8786
distributed.scheduler - INFO -   dashboard at:                    :33251
distributed.scheduler - INFO - Receive client connection: Client-80d379b6-f85e-11ed-ba3c-0ad1f44ba439
distributed.core - INFO - Starting established connection


In [4]:
import statsmodels
print(statsmodels.__version__)

0.13.1


In [5]:
from statsmodels.tsa.api import ARDL

/home/hadoop/miniconda/lib/python3.8/site-packages/statsmodels/compat/pandas.py:65: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import Int64Index as NumericIndex


In [21]:
from statsmodels.tsa.api import UECM

In [6]:
import dask.dataframe as dd

/home/hadoop/miniconda/lib/python3.8/site-packages/dask/dataframe/backends.py:187: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  _numeric_index_types = (pd.Int64Index, pd.Float64Index, pd.UInt64Index)
/home/hadoop/miniconda/lib/python3.8/site-packages/dask/dataframe/backends.py:187: FutureWarning: pandas.Float64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  _numeric_index_types = (pd.Int64Index, pd.Float64Index, pd.UInt64Index)
/home/hadoop/miniconda/lib/python3.8/site-packages/dask/dataframe/backends.py:187: FutureWarning: pandas.UInt64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  _numeric_index_types = (pd.Int64Index, pd.Float64Index, pd.UInt64Index)


## Data

In [7]:
data = dd.read_parquet("s3://trade-final-project-bucket/dataset/trades_quarter.parquet",
                     engine='pyarrow').persist()

/home/hadoop/miniconda/lib/python3.8/site-packages/charset_normalizer/api.py:105: UserWarning: Trying to detect encoding from a tiny portion of (19) byte(s).
  warn('Trying to detect encoding from a tiny portion of ({}) byte(s).'.format(length))


In [8]:
data.head()

distributed.scheduler - INFO - Register worker <WorkerState 'tcp://172.31.16.211:34749', name: dask.worker_0, memory: 0, processing: 91>
distributed.scheduler - INFO - Starting worker compute stream, tcp://172.31.16.211:34749
distributed.core - INFO - Starting established connection
distributed.scheduler - INFO - Register worker <WorkerState 'tcp://172.31.20.51:38711', name: dask.worker_1, memory: 0, processing: 0>
distributed.scheduler - INFO - Starting worker compute stream, tcp://172.31.20.51:38711
distributed.core - INFO - Starting established connection


,country,year_quarter,export_import,hs2,hs6,hs9,value_log,ex_log,gdp_log
0,230,2000_4,1,85,851110,851110100,10.934677,1.276011,10.493925
1,304,2001_2,1,60,600291,600291000,9.661289,-4.803976,14.789976
2,304,2002_1,2,63,630222,630222090,7.433667,-4.875197,14.807233
3,217,2003_4,2,61,611030,611030014,10.729044,NaN,10.521125
4,231,2004_1,1,84,842810,842810900,7.318540,-3.498025,10.910666


In [9]:
len(data)

13002864

In [10]:
data_d = data.dropna()

In [11]:
hs2_lst = data_d["hs2"].unique().compute()

In [12]:
len(hs2_lst)

88

In [22]:
def fit_ardl(data, meta=None):
    if data.isna().values.any():
        return "fail"
    model = UECM(data.value_log, 4, data[["ex_log","gdp_log"]], 4, fixed=data[["country"]])
    result = model.fit()
    
    # Return the fitted model result
    return result

In [23]:
results = dict()

for hs2 in hs2_lst:
    df = data_d[(data_d["hs2"]==hs2)&(data_d["export_import"]==1)].compute()
    df.sort_values('year_quarter', inplace=True)
    model = fit_ardl(df)
    results[hs2] = model 

/home/hadoop/miniconda/lib/python3.8/site-packages/statsmodels/tsa/base/tsa_model.py:590: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  warnings.warn('An unsupported index was provided and will be'
/home/hadoop/miniconda/lib/python3.8/site-packages/statsmodels/tsa/base/tsa_model.py:590: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  warnings.warn('An unsupported index was provided and will be'
/home/hadoop/miniconda/lib/python3.8/site-packages/statsmodels/tsa/base/tsa_model.py:590: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  warnings.warn('An unsupported index was provided and will be'
/home/hadoop/miniconda/lib/python3.8/site-packages/statsmodels/tsa/base/tsa_model.py:590: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  warnings.warn('An unsupported index was provided and will be'
/home/hadoop/minicon

ValueError: zero-size array to reduction operation maximum which has no identity

In [24]:
for i,k in results.items():
    print(i, k.summary())

85                               UECM Model Results                              
Dep. Variable:            D.value_log   No. Observations:               623315
Model:                  UECM(4, 4, 4)   Log Likelihood            -1404346.061
Method:               Conditional MLE   S.D. of innovations              9.998
Date:                Mon, 22 May 2023   AIC                        2808726.121
Time:                        05:43:18   BIC                        2808918.949
Sample:                             4   HQIC                       2808780.215
                               623315                                         
                     coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------
const              4.8223      0.046    104.165      0.000       4.732       4.913
value_log.L1      -0.8858      0.002   -368.691      0.000      -0.890      -0.881
ex_log.L1         -0.2577      0.

In [25]:
for i,k in results.items():
    print(i, k.ci_summary())

85                               Cointegrating Vector                              
                   coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------
const           -5.4441      0.050   -107.866      0.000      -5.543      -5.345
value_log.L1     1.0000          0        nan        nan       1.000       1.000
ex_log.L1        0.2909      0.004     77.449      0.000       0.284       0.298
gdp_log.L1      -0.2744      0.003    -87.681      0.000      -0.281      -0.268
60                               Cointegrating Vector                              
                   coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------
const           -7.6105      0.192    -39.591      0.000      -7.987      -7.234
value_log.L1     1.0000          0        nan        nan       1.000       1.000
ex_log.L1        0.004

In [45]:
param_lst = []

for i, k in results.items():
    param = k.params
    param_lst.append(param)

In [47]:
param_df = pd.DataFrame(param_lst)

In [49]:
param_df.index = [i for i in results.keys()]

In [50]:
param_df.head()

,const,value_log.L1,ex_log.L1,gdp_log.L1,D.value_log.L1,D.value_log.L2,D.value_log.L3,D.ex_log.L0,D.ex_log.L1,D.ex_log.L2,D.ex_log.L3,D.gdp_log.L0,D.gdp_log.L1,D.gdp_log.L2,D.gdp_log.L3,country
85,4.822308,-0.885779,-0.257657,0.243075,-0.087159,-0.057586,-0.029039,-0.410351,-0.113300,-0.078842,-0.036977,0.497975,0.190132,0.129951,0.063555,-0.001706
60,7.419540,-0.974907,-0.004681,0.043731,-0.023999,-0.015673,-0.011874,-0.063282,-0.040516,-0.022557,-0.016020,0.131019,0.066779,0.038641,0.021721,-0.001612
63,6.029533,-0.982154,-0.087237,0.097911,-0.012417,-0.004938,0.002842,-0.104533,-0.019060,-0.018371,-0.002669,0.139593,0.037859,0.031037,0.007695,-0.001153
84,4.292249,-0.931604,-0.269945,0.308197,-0.051175,-0.034948,-0.017194,-0.376095,-0.080410,-0.052498,-0.026239,0.472329,0.123769,0.081593,0.041392,-0.001084
64,5.717063,-0.913437,-0.032727,0.080660,-0.073234,-0.054354,-0.032341,-0.076555,-0.035639,-0.014681,-0.008961,0.174579,0.072127,0.042208,0.020557,-0.001780


In [51]:
import pickle

fileObj = open('data.obj', 'wb')
pickle.dump(results,fileObj)
fileObj.close()

In [ ]:
# https://blog.ms-researchhub.com/2021/04/23/estimating-long-run-coefficients-from-an-ardl-model/